In [84]:
import pandas as pd
import sqlite3

In [85]:
# Устанавливаем подключение к базе данных SQLite
conn = sqlite3.connect('indicators_db.sqlite3')

# Выполняем SQL-запрос и считываем результат в датафрейм
query = "SELECT * FROM org_indicators_2019"
df_2019 = pd.read_sql_query(query, conn)
query = "SELECT * FROM org_indicators_2020_2022"
df_2020 = pd.read_sql_query(query, conn)

def collapse_columns(df):
    df = df.copy()
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.to_series().apply(lambda x: "_".join([str(i) for i in x]))
    return df

# Закрываем соединение с базой данных
conn.close()

df_2019 = df_2019.pivot(index="ID", values="VALUE", columns=["YEAR", "INDICATOR_NAME"])
df_2019 = df_2019.reset_index()
df_2019 = collapse_columns(df_2019)
df_2019 = df_2019.rename(columns={"ID_": "ID"}) 
df_2020 = df_2020.pivot(index="ID", values="VALUE", columns=["YEAR", "INDICATOR_NAME"])
df_2020 = df_2020.reset_index()
df_2020 = collapse_columns(df_2020)
df_2020 = df_2020.rename(columns={"ID_": "ID"}) 
# Выводим содержимое датафрейма
print(df_2019)
print(df_2020)

          ID  2019_Число сотрудников  2019_Оценка на Яндекс  2019_Налоги   
0      19630                     7.0                    3.0          NaN  \
1      19643                     NaN                    2.0          NaN   
2      19647                    14.0                    3.0          NaN   
3      19657                     NaN                    1.0     659466.0   
4      19677                     NaN                    NaN     359672.0   
...      ...                     ...                    ...          ...   
9995  375884                     7.0                    1.0     353090.0   
9996  375933                     NaN                    2.0     241986.0   
9997  375954                    17.0                    2.0          NaN   
9998  375956                    17.0                    NaN          NaN   
9999  375960                     NaN                    2.0          NaN   

      2019_Выручка  2019_Оценка на 2GIS  2019_Проверок  
0       20522186.0            

In [86]:
main = pd.read_csv("main.csv")

In [87]:
df_1 = pd.merge(main, df_2020, how="left", on="ID")

In [91]:
df_1_1 = df_1[["ID", "Name", "Address", "2020_Выручка", "2021_Выручка", "2022_Выручка", "2020_Число сотрудников", "2021_Число сотрудников", "2022_Число сотрудников"]]

In [92]:
df_1_1["Дельта по выручке"] = df_1_1["2022_Выручка"] - df_1_1["2020_Выручка"]
df_1_1["Дельта по сотрудникам"] = df_1_1["2022_Число сотрудников"] - df_1_1["2020_Число сотрудников"]

/var/folders/sr/p3kpr51n4hs1q5w3zd7p_j0c0000gn/T/ipykernel_39835/3696649753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1_1["Дельта по выручке"] = df_1_1["2022_Выручка"] - df_1_1["2020_Выручка"]
/var/folders/sr/p3kpr51n4hs1q5w3zd7p_j0c0000gn/T/ipykernel_39835/3696649753.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1_1["Дельта по сотрудникам"] = df_1_1["2022_Число сотрудников"] - df_1_1["2020_Число сотрудников"]


In [94]:
df_1_1 = df_1_1.dropna(subset=["Дельта по выручке", "Дельта по сотрудникам"])

In [99]:
df_1_1 = df_1_1.sort_values(["Дельта по выручке", "ID"], ascending=[False, True])

In [106]:
df_1_1= df_1_1[['ID', 'Name', 'Address', '2020_Выручка', '2021_Выручка', '2022_Выручка','Дельта по выручке', 
            '2020_Число сотрудников', '2021_Число сотрудников', '2022_Число сотрудников', 'Дельта по сотрудникам']]
df_1_1.columns = ['Идентификатор', 'Наименование', 'Адрес', 'Выручка 2020', 'Выручка 2021', 'Выручка 2022','Дельта по выручке', 
            'Число сотрудников 2020', 'Число сотрудников 2021', 'Число сотрудников 2022', 'Дельта по сотрудникам']

In [128]:
df_1_1.to_csv("df_2.csv", index=False)

In [143]:
list = [357706, 198181, 27549, 217895, 201139, 357706, 20105]
list = pd.Series(list)
list_2 = pd.read_excel("Выручка_по_компаниям.xlsx", sheet_name="Лист1")

In [144]:
list_2 = pd.concat([list_2["Айдии"], list], ignore_index=True)

In [149]:
list_2 = pd.DataFrame(data=list_2)
list_2 = list_2.rename(columns={0 : "ID"})

In [150]:
df_2 = pd.merge(list_2, df_2020, how="left")

In [154]:
df_2 = df_2[['ID', '2020_Выручка', '2021_Выручка', '2022_Выручка']]

In [157]:
df_2['2022_Выручка'] = df_2['2022_Выручка'].combine_first(df_2['2020_Выручка'])

In [160]:
df_2 = df_2.drop_duplicates()

In [162]:
df_2.sum() + 22495241

ID               25914059.0
2020_Выручка    268354937.0
2021_Выручка    300355984.0
2022_Выручка    462125109.0
dtype: float64

In [187]:
df_3_2019 = df_2019[["ID", "2019_Число сотрудников"]]

In [188]:
df_3_2020 = df_2020[["ID", "2020_Число сотрудников"]]

In [189]:
df_3 = pd.merge(df_3_2019, df_3_2020, how="outer")

In [190]:
df_3["2020_Число сотрудников"] = df_3["2020_Число сотрудников"].combine_first(df_3["2019_Число сотрудников"])

In [191]:
df_3 = df_3[df_3["2020_Число сотрудников"] >= 15]

In [192]:
df_3 = df_3.drop_duplicates()

In [193]:
df_3 = df_3.drop(columns="2019_Число сотрудников")

In [194]:
df_3.columns = ['Идентификатор', 'Сотрудники 2019_2020']
df_3['Сотрудники 2019_2020'] = df_3['Сотрудники 2019_2020'].astype(int)

In [195]:
df_3 = df_3.sort_values(["Сотрудники 2019_2020", "Идентификатор"], ascending=[True, False])

In [196]:
df_3.to_csv("df_3.csv", index=False)

In [214]:
df_2019_nalog = df_2019[["ID", "2019_Налоги"]]

In [215]:
df_2020_nalog = df_2020[["ID", "2020_Налоги", "2021_Налоги", "2022_Налоги"]]

In [216]:
df_4 = pd.merge(main[["ID", "Name", "Address"]], df_2019_nalog, how="outer")

In [217]:
df_4 = pd.merge(df_4, df_2020_nalog, how="outer")

In [218]:
df_4 = df_4.dropna()

In [219]:
df_4["Налоги (Всего)"] = df_4["2019_Налоги"] + df_4["2020_Налоги"] + df_4["2021_Налоги"] + df_4["2022_Налоги"]

In [220]:
df_4 = df_4.sort_values("Налоги (Всего)", ascending=False)
df_4 = df_4.drop_duplicates()
df_4.columns = ['Идентификатор', 'Наименование', 'Адрес', 'Налоги (2019 г.)', 'Налоги (2020 г.)', 'Налоги (2021 г.)', 'Налоги (2022 г.)', "Налоги (Всего)"]

In [221]:
df_4[:10].to_csv("df_4.csv", index=False)